In [1]:

import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
from tqdm import tqdm
import pandas as pd
import os

In [2]:
data = pd.read_csv('./dataset/org/H1_nav.csv')

In [3]:
ignore_columns = ['Participant', 'Dataset']
features = [col for col in data.columns if col not in ignore_columns]

In [4]:
device = torch.device('cuda')
device
data.head()

,FrameNumber,x,y,z,HMDRX,HMDRY,HMDRZ,Participant,Dataset,ViewFrame
0,1,0.0500,1.7868,-1.0947,6.9163,350.8206,359.9912,P01_V1,H1,1
1,2,0.0498,1.7871,-1.0951,6.9116,351.1272,0.0094,P01_V1,H1,2
2,3,0.0498,1.7872,-1.0955,6.8915,351.3081,0.0221,P01_V1,H1,3
3,4,0.0498,1.7870,-1.0961,6.9375,351.5385,359.9731,P01_V1,H1,4
4,5,0.0497,1.7865,-1.0969,7.1456,351.7421,359.8803,P01_V1,H1,5


In [5]:
data = data.fillna(method='ffill').fillna(method='bfill')
data = data.drop(columns=ignore_columns)

In [6]:
class Timeseries_Dataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray, seq_len: int = 32):
        self.X = torch.tensor(X).float()
        self.y = torch.tensor(y).float()
        self.seq_len = seq_len

    def __len__(self):
        return self.X.__len__() - (self.seq_len - 1)

    def __getitem__(self, index):
        return {'x': torch.tensor(self.X[index:index + self.seq_len], dtype=torch.float),
                'y': torch.tensor(self.y[index + self.seq_len - 1], dtype=torch.long)}


In [9]:
class LSTMClassifier(nn.Module):

    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.to(device) for t in (h0, c0)]



In [7]:
batch_size = 200
lr = 0.0005
input_dim = 7
hidden_dim = 256
layer_dim = 5
output_dim = 2
seq_dim = 32
target_column = 'x'

feature_columns = data.columns[~data.columns.isin([target_column])]
train, validation = data[:int(len(data) * 0.8)], data[int(len(data) * 0.2):]
train_features, train_target = train[feature_columns], train[[target_column]]
validation_features, validation_target = validation[feature_columns], validation[[target_column]]
train_dataset = Timeseries_Dataset(X=train_features.values, y=train_target.values, seq_len=seq_dim)
validation_dataset = Timeseries_Dataset(X=validation_features.values, y=validation_target.values, seq_len=seq_dim)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)



In [10]:
weight = '/kaggle/input/weight-lstm/best_30.pth'

phase_training = True
if os.path.exists(weight):
    phase_training = False

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = LSTMClassifier(input_dim, hidden_dim, layer_dim, output_dim)
model = model.to(device)
